In [5]:
import pandas as pd
import networkx as nx
import scipy as sp
import numpy as np
from scipy import sparse

df = pd.read_csv("./data/edges.op", sep=" ")
orig_graph = nx.from_pandas_dataframe(df, source='node_from', target='node_to', edge_attr=['distance', 'animal_density', 'grid_cell_x', 'grid_cell_y'])
# orig_graph = nx.subgraph(orig_graph, nx.node_connected_component(orig_graph, 0))
new_graph, dist, scores, adj = create_extended_graph(orig_graph)
# dist = nx.floyd_warshall_numpy(new_graph, weight='distance')

In [20]:
with open('./data/dist_ext.gop', 'w') as f:
    # f.write(str(len(new_graph.nodes())) + " 1 0 0\n")
    f.write(str(len(dist)) + " 1 0 0\n")
with open('./data/dist_ext.gop', 'ab') as f:
    for line in np.matrix(dist):
        np.savetxt(f, line, fmt='%.2f')
with open('./data/dist_ext.gop', 'a') as f:
    # for node in new_graph.nodes_iter():
    #     f.write(str(nx.get_node_attributes(new_graph, 'animal_density')[node])+"\n")
    # for score in scores:
	 #    f.write(str(score) + "\n")
	for i in range(len(dist)):
		try:
			f.write(str(nx.get_node_attributes(new_graph, 'animal_density')[i])+"\n")
		except:
			f.write(str(0)+"\n")

In [26]:
# print(nx.shortest_path_length(orig_graph, 0, 15, weight="distance"))
# print(nx.shortest_path_length(new_graph, 0, 15, weight="distance"))
# print(dist[0,15])
# print(len(new_graph.edges()))
# print(len(new_graph.nodes()))
# print(nx.adjacency_matrix(new_graph, weight='distance')[0, 553])
# print(nx.adjacency_matrix(orig_graph, weight='distance')[0, 15])
# print(adj[0, 1])
# print(nx.neighbors(orig_graph, 15))
# print(nx.neighbors(new_graph, 554))
# print(orig_graph[65][5])
# print(dist[5,65])
# print(dist[5,554])
# print(nx.get_node_attributes(new_graph, 'animal_density')[554])
print(orig_graph[19][15]['animal_density'])

8457.48900607


In [2]:
def create_extended_graph(orig_graph):
	new_graph = nx.Graph()
	scores = []
	i = 553
	# i = len(orig_graph.nodes())
	adj = np.zeros((3059, 3059))
	adj[:] = np.inf
	for edge in orig_graph.edges():
		new_graph.add_node(edge[0], animal_density=0, grid_cell_x=orig_graph[edge[0]][edge[1]]['grid_cell_x'],
		                   grid_cell_y=orig_graph[edge[0]][edge[1]]['grid_cell_y'])
		new_graph.add_node(edge[1], animal_density=0, grid_cell_x=orig_graph[edge[0]][edge[1]]['grid_cell_x'],
		                   grid_cell_y=orig_graph[edge[0]][edge[1]]['grid_cell_y'])
		new_graph.add_node(i, animal_density=orig_graph[edge[0]][edge[1]]['animal_density'],
		                   grid_cell_x=orig_graph[edge[0]][edge[1]]['grid_cell_x'],
		                   grid_cell_y=orig_graph[edge[0]][edge[1]]['grid_cell_y'])
		new_graph.add_edge(edge[0], i, distance=float(orig_graph[edge[0]][edge[1]]['distance']) / 2,
		                   grid_cell_x=orig_graph[edge[0]][edge[1]]['grid_cell_x'],
		                   grid_cell_y=orig_graph[edge[0]][edge[1]]['grid_cell_y'])
		new_graph.add_edge(i, edge[1], distance=float(orig_graph[edge[0]][edge[1]]['distance']) / 2,
		                   grid_cell_x=orig_graph[edge[0]][edge[1]]['grid_cell_x'],
		                   grid_cell_y=orig_graph[edge[0]][edge[1]]['grid_cell_y'])
		adj[int(edge[0])][i] = float(orig_graph[edge[0]][edge[1]]['distance']) / 2
		adj[i][int(edge[1])] = float(orig_graph[edge[0]][edge[1]]['distance']) / 2
		adj[i][int(edge[0])] = adj[int(edge[0])][i]
		adj[int(edge[1])][i] = adj[i][int(edge[1])]
		i += 1
	for node_num, node_data in new_graph.nodes(data=True):
		# print(node_num)
		scores.append(node_data['animal_density'])

	# dist = nx.floyd_warshall_numpy(new_graph, weight='distance').tolist()
	# adj = nx.adjacency_matrix(new_graph, weight='distance')
	dist = sp.sparse.csgraph.shortest_path(adj, directed=False)
	return new_graph, dist, scores, adj

In [3]:
def get_adj_matrix(df):
	adj = np.zeros((553, 553))

	for index, row in df.iterrows():
		adj[int(row['node_from'])][int(row['node_to'])] = row['distance']
		adj[int(row['node_to'])][int(row['node_from'])] = row['distance']
	return adj

In [8]:
import networkx as nx

gr = nx.Graph()
# gr.add_node(5, animal_density=10, grid_cell_x=0, grid_cell_y=0)
# gr.add_node(5, animal_density=12, grid_cell_x=0, grid_cell_y=0)
gr.add_edge(4, 5, animal_density=10, distance=1, grid_cell_x=0, grid_cell_y=0)
gr.add_edge(5, 6, animal_density=10, distance=1, grid_cell_x=0, grid_cell_y=0)
gr.add_edge(6, 7, animal_density=10, distance=1, grid_cell_x=0, grid_cell_y=0)
new_graph, dist, score = create_extended_graph(gr)
new_graph.nodes(data=True)

[(100000, {'animal_density': 10, 'grid_cell_x': 0, 'grid_cell_y': 0}),
 (100001, {'animal_density': 10, 'grid_cell_x': 0, 'grid_cell_y': 0}),
 (100002, {'animal_density': 10, 'grid_cell_x': 0, 'grid_cell_y': 0}),
 (4, {'animal_density': 0, 'grid_cell_x': 0, 'grid_cell_y': 0}),
 (5, {'animal_density': 0, 'grid_cell_x': 0, 'grid_cell_y': 0}),
 (6, {'animal_density': 0, 'grid_cell_x': 0, 'grid_cell_y': 0}),
 (7, {'animal_density': 0, 'grid_cell_x': 0, 'grid_cell_y': 0})]

In [9]:
import numpy as np

with open('./data/dist_ext_small.gop', 'w') as f:
    f.write(str(len(new_graph.nodes())) + " 1 0 0\n")
with open('./data/dist_ext_small.gop', 'ab') as f:
    for line in np.matrix(dist):
        np.savetxt(f, line, fmt='%.2f')
with open('./data/dist_ext_small.gop', 'a') as f:
    for node in new_graph.nodes_iter():
        f.write(str(nx.get_node_attributes(new_graph, 'animal_density')[node])+"\n")